# Monthly Keyword Analysis 

This notebook ingests the preprocessed data from `../interim/text` downloaded by `download_datasets.ipynb` and uses a TF-IDF method to identify the top 10 keywords for each month. This is done by implementing the following procedure: For each month, we train and fit a separate TF-IDF model, then collect the top 10 scoring words for each email and sum their occurrences to identify the top 10 most frequently occurring keywords for each month.     

Finally, the data is saved as a single csv file and pushed to remote storage for visualization with Superset. 

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

from pathlib import Path

from dotenv import load_dotenv
load_dotenv("../../.env")

import sys
sys.path.append('../..')
from src import utils

In [2]:
BASE_PATH = os.getenv("LOCAL_DATA_PATH", "../../data/")

In [3]:
df = utils.load_dataset(f"{BASE_PATH}interim/text/")
df.head()

,Message-ID,Date,Body
0,<1519862707.18745.0@posteo.de>,"Wed, 28 Feb 2018 18:05:07 -0600",['\nI have an update here:\n\nhttps://bodhi.fe...
1,<b88b4d3b-b8f7-2ea6-ec41-ab572b831717@dustymab...,"Wed, 28 Feb 2018 19:43:15 -0500","[""-----BEGIN PGP SIGNED MESSAGE-----\nHash: SH..."
2,<45117c81-43ff-656d-85c3-3cf003bd0d14@fedorapr...,"Wed, 28 Feb 2018 20:49:25 -0500","['On 02/28/2018 07:05 PM, mcatanzaro(a)gnome.o..."
3,<CAA55FSN-R4oV0os0LihZQTp8aa0NkR9jQPh44subK2+9...,"Wed, 28 Feb 2018 21:11:19 -0500","['On 28 February 2018 at 10:03, Nicolas Mailho..."
4,<p77nrn$t3b$1@blaine.gmane.org>,"Thu, 01 Mar 2018 03:19:34 +0100","['Fabio Valentini wrote:\n> AFAICT, those ""bro..."


In [4]:
df.shape

(41997, 3)

## Text Preprocessing

Due to the casual nature of email writing, along with some known useless artifacts present in our textual dataset, we need to clean our data a bit before performing our analysis. 

In [5]:
def strip_thread(text):
    text = text.replace("\r", "")
    lines = text.split("\n")
    lines = [l for l in lines if len(l) > 0]
    lines = [line for line in lines if line[0] != ">"]
    lines = [line for line in lines if line[:3] != "Re:"]
    lines = [line for line in lines if line[:7] != "Subject"]
    lines = [line for line in lines if line[:5] != "From:"]
    lines = [line for line in lines if line[:5] != "Date:"]
    lines = [line for line in lines if "BEGIN PGP SIGNED MESSAGE" not in line]
    lines = [line for line in lines if line[:5] != "Hash:"]
    lines = [line for line in lines if line[:10] != "Version: G"]
    lines = [line for line in lines if "wrote:" not in line]
    lines = [line for line in lines if "wrote :" not in line]
    lines = [line for line in lines if "writes:" not in line]
    lines = [line for line in lines if line[:7] != "Am Mit,"]
    lines = [line for line in lines if line[:7] != "Am Don,"]
    lines = [line for line in lines if line[:7] != "Am Mon,"]
    lines = [line for line in lines if line[:7] != "Quoting"]
    lines = [line for line in lines if line[:10] != "Em Quinta,"]
    lines = [line for line in lines if "said:" not in line]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), .. (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec) 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday) .. (January|February|March|April|May|June|July|August|September|October|November|December) 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec) .., 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), 20[\d]{2}-[\d]{2}-[\d]{2} at.*", line) == None]
    lines = [line for line in lines if line[-6:] != "said: "]
    lines = [line for line in lines if line[-8:] != "babbled:"]
    lines = [line for line in lines if line[-7:] != "wrot=e:"]
    lines = [line for line in lines if line[-8:] != "A9crit :"]
    lines = [line for line in lines if line[0] != "|"]
    return "\n".join(lines)

#format for CSV, clean special characters, and remove extranous emails 
def pandas_clean(emails):
    emails['Body'].replace(to_replace=[r"\n", "\n",], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace=[r"\'", "\'",">",'<',"= ", "-", "http\S+" ], value="", regex=True, inplace=True)
    emails['Body'].replace(to_replace=[ r"\\\s+", "\\\s+" , "="], value= "", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace= ["_","3D"], value= "", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    emails['Body'] = emails["Body"].apply(lambda x: x.strip().replace(r'\n',''))
    
    emails.drop(emails.index[emails['Body'] == ''], inplace = True)
    emails.drop(emails.index[emails['Body'] == ' '], inplace = True)
    emails.dropna(subset=['Body'], inplace=True)
    
    emails = emails.reset_index()
    emails.drop('index',axis=1, inplace=True )
    return emails

In [6]:
clean = df.copy()
clean["Body"]  = clean["Body"].apply(strip_thread)
clean = pandas_clean(clean)
clean

,Message-ID,Date,Body
0,<1519862707.18745.0@posteo.de>,"Wed, 28 Feb 2018 18:05:07 -0600",[I have an update here: is blocked due to some...
1,<1519885501.29070.9.camel@fedoraproject.org>,"Thu, 01 Mar 2018 01:25:01 -0500","[""C2A0Following is the list of topics that wil..."
2,<1519886671.29070.13.camel@fedoraproject.org>,"Thu, 01 Mar 2018 01:44:31 -0500","[""C2A0The Fedora Packaging Committee has some ..."
3,<1519888291.16369.3.camel@fedoraproject.org>,"Thu, 01 Mar 2018 02:11:31 -0500","[""C2A0The Fedora Packaging Committee has some ..."
4,<20180301103525.22531.58685@mailman01.phx2.fed...,"Thu, 01 Mar 2018 10:35:25 +0000","[Hi,when compiling qarte4.4.0 with this spec f..."
...,...,...,...
13862,<20190630045144.22906.88878@mailman01.phx2.fed...,"Sun, 30 Jun 2019 04:51:44 +0000","[""speedtestcli and python3speedtestcli appear ..."
13863,<c94d58e2-e218-8f69-0ff2-a46cd57ab65b@redhat.com>,"Sun, 30 Jun 2019 11:46:31 +0200","[Hey,pip and Python packages generally deem 5...."
13864,<ee3d5633-fd1b-7f71-e173-788c61c9a9e3@gmail.com>,"Sun, 30 Jun 2019 05:21:47 -0500","[""Hi,Does Fedora Silverblue Rawhide not suppor..."
13865,<ee5805d4ea36e9b22893b21d2cf41bfda2db6dca.came...,"Sun, 30 Jun 2019 08:51:21 -0700","[""Hi folks! Im proposing we cancel the QA meet..."


In [7]:
clean["Date"] = clean["Date"].apply(lambda x: pd.to_datetime(x))
clean["Chunk"] = clean["Date"].apply(lambda x: datetime.date(x.year,x.month,1) )
clean = clean.sort_values(by="Date")
clean.reset_index(inplace=True, drop=True)
clean.head()

,Message-ID,Date,Body,Chunk
0,<20180101220004.0E97560A400C@fedocal02.phx2.fe...,2018-01-01 22:00:04+00:00,"[Dear all,You are kindly invited to the meetin...",2018-01-01
1,<20180101220004.0632660A400B@fedocal02.phx2.fe...,2018-01-01 22:00:04+00:00,"[Dear all,You are kindly invited to the meetin...",2018-01-01
2,<20180101221314.GA52721@rawhide-composer.phx2....,2018-01-01 22:13:15+00:00,[OLD: FedoraRawhide20171231.n.0NEW: FedoraRawh...,2018-01-01
3,<20180101233509.D734E60478E3@bastion01.phx2.fe...,2018-01-01 23:35:09+00:00,[Missing expected images:Server dvd i386Workst...,2018-01-01
4,<66075732-52f6-2eb8-de1b-d89ec18244db@redhat.com>,2018-01-02 10:26:51+01:00,"[""Could you please drop the dependency on GCC ...",2018-01-01


In [8]:
clean.tail()

,Message-ID,Date,Body,Chunk
13862,<20201130211635.21006.77071@mailman01.iad2.fed...,2020-11-30 21:16:35+00:00,[Hello. I want to buy myself Dell XPS 13 9310 ...,2020-11-01
13863,<28998e59-9857-8c3d-548f-47d6642d3789@redhat.com>,2020-11-30 23:00:32+01:00,"[Hi guys,not sure whether I missed something, ...",2020-11-01
13864,<e1fcf02c-43d0-00e1-6b6a-0eed22468ca0@redhat.com>,2020-11-30 14:06:04-08:00,"[Hi,As part of the f34 change request[1] for r...",2020-11-01
13865,<637e6814-bc05-4713-6226-7192a437df46@redhat.com>,2020-11-30 23:23:18+01:00,[Dne 30. 11. 20 v 19:29 Rudolf Kastl napsal(a)...,2020-11-01
13866,<1fb0c88e-79fa-ce05-8c77-5a0e8c95a0a8@redhat.com>,2020-11-30 23:31:39+01:00,[Dne 30. 11. 20 v 23:06 Tom Stellard napsal(a)...,2020-11-01


## Single month example 

Here we will prototype our method for identifying top N key words for a single month, to ensure it works properly before applying it to the entire dataset. 

In [9]:
corpus = clean[clean.Chunk == datetime.date(2020,11,1)].Body
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [10]:
X.shape

(440, 37733)

In [11]:
feature_array = np.array(vectorizer.get_feature_names())

Document = []
for i, j in enumerate(X[0].toarray()[0]):
    if j > 0:
        Document.append((i,j))
        
top_10 = sorted(Document, key = lambda x:x[1],reverse=True)[0:10]
top_10_keys = [x[0] for x in top_10]
[feature_array[i] for i in top_10_keys]

['cloudbaseqcow2qcow2',
 'ttest',
 'aarch64',
 'x8664',
 'uefiurl',
 'fedoracloud3320201031',
 'soft',
 '712248',
 '712249',
 '712250']

In [12]:
corpus[0:1]

13427    [No missing expected images.Soft failed openQA...
Name: Body, dtype: object

In [13]:
corpus[corpus[0:1].index[0]]

'[No missing expected images.Soft failed openQA tests: 1/7 (x8664), 1/7 (aarch64)(Tests completed, but using a workaround for a known bug)Old soft failures (same test soft failed in FedoraCloud3320201031.0):ID: 712253\\tTest: x8664 CloudBaseqcow2qcow2 cloudautocloudURL: 712261\\tTest: aarch64 CloudBaseqcow2qcow2 cloudautocloud(a)uefiURL: openQA tests: 6/7 (x8664), 6/7 (aarch64)New passes (same test not passed in FedoraCloud3320201031.0):ID: 712248\\tTest: x8664 CloudBaseqcow2qcow2 baserebootunmountURL: 712249\\tTest: x8664 CloudBaseqcow2qcow2 basesystemloggingURL: 712250\\tTest: x8664 CloudBaseqcow2qcow2 baseupdatecliURL: 712251\\tTest: x8664 CloudBaseqcow2qcow2 baseservicemanipulationURL: 712252\\tTest: x8664 CloudBaseqcow2qcow2 baseservicesstartURL: 712254\\tTest: x8664 CloudBaseqcow2qcow2 baseselinuxURL: 712255\\tTest: aarch64 CloudBaseqcow2qcow2 basesystemlogging(a)uefiURL: 712256\\tTest: aarch64 CloudBaseqcow2qcow2 baserebootunmount(a)uefiURL: 712257\\tTest: aarch64 CloudBaseqcow2

Looks out key words are reasonable given the email in question above.   

### Run full analysis entire dataset 

Now that we are confident our approach works, we will break it up into manageable functions and apply it to each months dataset. 

In [14]:
def train_monthly_tfidf(corpus):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer


def top_words_per_email(email_vector,feature_array, top_words=10):
    Document = []
    for i, j in enumerate(email_vector.toarray()[0]):
        if j > 0:
            Document.append((i,j))
    top_N = sorted(Document, key = lambda x:x[1],reverse=True)[0:top_words]
    top_N_keys = [x[0] for x in top_N]
    top_N_words = [feature_array[i] for i in top_N_keys]
    return top_N_words


def get_monthly_keywords(corpus, chunk):
    X, vectorizer = train_monthly_tfidf(corpus)
    feature_array = np.array(vectorizer.get_feature_names())
    keywords=[]
    for i in range(X.shape[0]):
        keywords.extend(top_words_per_email(X[i],feature_array))
        
    keywords = Counter(keywords).most_common(10)
    keywords = pd.DataFrame(keywords, columns=["word","count"])
    keywords["month"] = chunk

    return keywords

In [15]:
# For each document collect the top 10 words, then sum the top 10 for each month.

months = clean.Chunk.unique()
monthly_keywords = pd.DataFrame([],columns=["word","count","month"])

for month in months:
    corpus = clean[clean.Chunk == month].Body
    monthly_keywords = monthly_keywords.append(get_monthly_keywords(corpus,month),
                                               ignore_index=True)
    print(month)

monthly_keywords = pd.DataFrame(monthly_keywords)

2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01


In [16]:
monthly_keywords = monthly_keywords.reset_index().set_index("word")

In [17]:
monthly_keywords = monthly_keywords.drop("index", axis=1)

In [18]:
monthly_keywords

,count,month
word,,
fc28,32,2018-01-01
test,31,2018-01-01
x8664,27,2018-01-01
pc9kaxy,26,2018-01-01
change,26,2018-01-01
...,...,...
cloudbaseqcow2qcow2,56,2020-11-01
soft,55,2020-11-01
test,39,2020-11-01


## Upload results to S3

In [19]:
new_files = (
    (monthly_keywords, f'{BASE_PATH}/processed/keywords.csv'),
)

In [20]:
Path(f'{BASE_PATH}/processed').mkdir(parents=True, exist_ok=True)

In [21]:
monthly_keywords.to_csv(new_files[0][1], header=False)

In [22]:
if os.getenv('RUN_IN_AUTOMATION'):
    utils.upload_files((f, f'processed/{Path(f).stem}/keywords.csv') for _, f in new_files)